In [5]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import csv

model = YOLO("yolov8x.pt")  

image_folder = r"C:\Users\mgree\Downloads\Images_Dataset_im"
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

csv_output_path = os.path.join(image_folder, "detection_results.csv")

with open(csv_output_path, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Image Name", "Number of Vehicles", "Detected Objects", "Confidence Scores", "Non-Objects"])

    if not image_files:
        print(f"No images found in folder: {image_folder}")
        exit()

    for image_name in image_files:
        image_path = os.path.join(image_folder, image_name)
        print(f"Processing {image_path}")

        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load {image_path}")
            continue

        results = model(image)
        result = results[0]

        vehicle_classes = {"car", "truck", "bus", "motorcycle"}
        detected_vehicles = []
        confidence_scores = []
        non_objects = []

        boxes = result.boxes  
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]  
            confidence = box.conf[0] 
            class_id = int(box.cls[0]) 

            if confidence > 0.5:  
                label = result.names[class_id]
                if label in vehicle_classes:
                    detected_vehicles.append(label)
                    confidence_scores.append(f"{confidence:.2f}")
                    color = (0, 255, 0)  
                else:
                    non_objects.append(label)
                    color = (0, 0, 255)  

                cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                cv2.putText(image, f"{label}: {confidence:.2f}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        csv_writer.writerow([
            image_name, 
            len(detected_vehicles),  
            ", ".join(detected_vehicles),  
            ", ".join(confidence_scores),  
            ", ".join(non_objects)  
        ])

        cv2.imshow("Object Detection", image)
        key = cv2.waitKey(0)  
        if key == 27:  
            break

cv2.destroyAllWindows()

print(f"Detection results saved to: {csv_output_path}")


Processing C:\Users\mgree\Downloads\Images_Dataset_im\AP1.jpg

0: 480x640 1 car, 2087.2ms
Speed: 16.8ms preprocess, 2087.2ms inference, 6.1ms postprocess per image at shape (1, 3, 480, 640)
Processing C:\Users\mgree\Downloads\Images_Dataset_im\AP10.jpg

0: 640x416 1 car, 1 truck, 830.0ms
Speed: 2.6ms preprocess, 830.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 416)
Processing C:\Users\mgree\Downloads\Images_Dataset_im\AP12.jpg

0: 640x480 5 cars, 1 bus, 962.6ms
Speed: 6.0ms preprocess, 962.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
Processing C:\Users\mgree\Downloads\Images_Dataset_im\AP13.jpg

0: 640x480 1 person, 1 car, 1 motorcycle, 911.4ms
Speed: 7.5ms preprocess, 911.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
Processing C:\Users\mgree\Downloads\Images_Dataset_im\AP24.jpg

0: 640x480 2 cars, 984.3ms
Speed: 8.8ms preprocess, 984.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
Processing C:\Us

In [6]:
import os
import cv2
import pytesseract
import re
import csv
import numpy as np
from ultralytics import YOLO

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def clean_license_plate_text(text):
    return re.sub(r"[^A-Z0-9\s\-]", "", text).strip()


vehicle_model = YOLO('yolov8x.pt')


image_folder = r"C:\Users\mgree\Downloads\Images_Dataset_det"
csv_output_path = os.path.join(image_folder, "detection_results.csv")


with open(csv_output_path, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Image Name", "Vehicles Found", "Number Plate", "Number of Vehicles Detected"])
    supported_formats = (".jpg", ".jpeg", ".png", ".bmp", ".tiff")
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(supported_formats)]

    if not image_files:
        print("No image files found in the specified folder.")
    else:
        for image_file in image_files:
            image_path = os.path.join(image_folder, image_file)
            print(f"Processing file: {image_path}")

            try:
                image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
            except Exception as e:
                print(f"Error reading file {image_file}: {e}")
                continue

            if image is None:
                print(f"Failed to load image: {image_path}. Skipping.")
                continue

            vehicle_results = vehicle_model(image)

           
            detected_vehicles = []
            detected_plates = []
            vehicle_count = 0

            for box in vehicle_results[0].boxes:
                cls = int(box.cls[0])  
                confidence = box.conf[0]  
                label = vehicle_results[0].names[cls]  

                if label in ["car", "truck", "bus", "motorcycle"] and confidence > 0.5:
                    x1, y1, x2, y2 = map(int, box.xyxy[0]) 
                    
                    if (x2 - x1) < 0.9 * image.shape[1] and (y2 - y1) < 0.9 * image.shape[0]:
                        vehicle_crop = image[y1:y2, x1:x2]

                        gray = cv2.cvtColor(vehicle_crop, cv2.COLOR_BGR2GRAY)
                        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
                        edges = cv2.Canny(blurred, 50, 200)

                        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                        for contour in contours:
                            x, y, w, h = cv2.boundingRect(contour)
                            aspect_ratio = w / float(h)

                            if 2.5 < aspect_ratio < 5 and 500 < w * h < 2000: 
                                plate_region = vehicle_crop[y:y+h, x:x+w]
                                raw_text = pytesseract.image_to_string(plate_region, config='--psm 8').strip()
                                cleaned_text = clean_license_plate_text(raw_text)

                                if len(cleaned_text) < 5:
                                    continue

                                detected_plates.append(cleaned_text)
                                print(f"Image: {image_file}, Detected license plate text: {cleaned_text}")
                                cv2.rectangle(vehicle_crop, (x, y), (x + w, y + h), (0, 255, 0), 2)
                                cv2.putText(vehicle_crop, cleaned_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                        detected_vehicles.append(label)
                        vehicle_count += 1
                        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
                        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

            csv_writer.writerow([image_file, ", ".join(detected_vehicles), ", ".join(detected_plates), vehicle_count])

            output_path = os.path.join(image_folder, f"output_{image_file}")
            cv2.imwrite(output_path, image)
            print(f"Processed image saved at: {output_path}")

            
            cv2.imshow("Processed Image", image)
            key = cv2.waitKey(0)
            if key == 27: 
                break

cv2.destroyAllWindows()


print(f"Detection results saved to: {csv_output_path}")


Processing file: C:\Users\mgree\Downloads\Images_Dataset_det\AP1.jpg

0: 480x640 1 car, 2210.9ms
Speed: 17.7ms preprocess, 2210.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Image: AP1.jpg, Detected license plate text: -APOSBY7739
Image: AP1.jpg, Detected license plate text: -APO5BY7739
Image: AP1.jpg, Detected license plate text: APOSBY
Processed image saved at: C:\Users\mgree\Downloads\Images_Dataset_det\output_AP1.jpg
Processing file: C:\Users\mgree\Downloads\Images_Dataset_det\AP12.jpg

0: 640x480 5 cars, 1 bus, 958.5ms
Speed: 6.0ms preprocess, 958.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)
Processed image saved at: C:\Users\mgree\Downloads\Images_Dataset_det\output_AP12.jpg
Processing file: C:\Users\mgree\Downloads\Images_Dataset_det\AP13.jpg

0: 640x480 1 person, 1 car, 1 motorcycle, 1053.9ms
Speed: 7.3ms preprocess, 1053.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
Image: AP13.jpg, Detected license plate te